# Data Preprocessing

## Carbon Monitor Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pandas requests

Considering the EU Carbon Monitor Dataset to keep the scope of the thesis limited to EU.

In [ ]:
import pandas as pd

# Loading the same dataset like in EDA
EU_data = pd.read_csv("/content/drive/MyDrive/Dataset/carbonmonitor-eu.csv")

# Converting date column to datetime format
EU_data['date'] = pd.to_datetime(EU_data['date'], dayfirst=True)

# Extracting year from date
EU_data['year'] = EU_data['date'].dt.year

EU_data.head()


,country,date,sector,MtCO2 per day,year
0,Norway,2019-01-01,Power,0.006542,2019
1,Norway,2019-01-02,Power,0.005956,2019
2,Norway,2019-01-03,Power,0.006048,2019
3,Norway,2019-01-04,Power,0.005940,2019
4,Norway,2019-01-05,Power,0.005649,2019


Yearly Aggregates

In [ ]:
# Grouping by country, sector, and year and then sum MtCO2 per day for each
yearly_sector_agg = EU_data.groupby(['country', 'sector', 'year'])['MtCO2 per day'].sum().reset_index()

# Renaming column for clarity
yearly_sector_agg.rename(columns={'MtCO2 per day': 'MtCO2 per year'}, inplace=True)

yearly_sector_agg.head()


,country,sector,year,MtCO2 per year
0,Austria,Domestic Aviation,2019,0.052956
1,Austria,Domestic Aviation,2020,0.043006
2,Austria,Domestic Aviation,2021,0.053115
3,Austria,Domestic Aviation,2022,0.032175
4,Austria,Domestic Aviation,2023,0.036633


In [ ]:
# Grouping by country and year only for total annual emissions
yearly_total_agg = yearly_sector_agg.groupby(['country', 'year'])['MtCO2 per year'].sum().reset_index()

yearly_total_agg.rename(columns={'MtCO2 per year': 'Total MtCO2 per year'}, inplace=True)
yearly_total_agg.head()


,country,year,Total MtCO2 per year
0,Austria,2019,70.303931
1,Austria,2020,65.951457
2,Austria,2021,72.662722
3,Austria,2022,72.706944
4,Austria,2023,71.303230


In [ ]:
# Merging sector data with total per country-year
combined_data = pd.merge(yearly_sector_agg, yearly_total_agg, on=['country', 'year'])

combined_data.head()


,country,sector,year,MtCO2 per year,Total MtCO2 per year
0,Austria,Domestic Aviation,2019,0.052956,70.303931
1,Austria,Domestic Aviation,2020,0.043006,65.951457
2,Austria,Domestic Aviation,2021,0.053115,72.662722
3,Austria,Domestic Aviation,2022,0.032175,72.706944
4,Austria,Domestic Aviation,2023,0.036633,71.303230


In [ ]:
# Saving the final combined sector-level + total emissions data
Carbon_Monitor = combined_data.copy()
Carbon_Monitor.to_csv("Carbon_Monitor_Final.csv", index=False)


# SDG's for Composite Index Building

Th SDG's listed below were choosen after studying works on Greenhouse gases emissons, Climate Monitor report. We aim to cover a minimum of 5-10 sdg features that can help us build a robust composite index. Each SDG related data was extracted from Eurostat portal using the API call. These were filtered and cleaned to fit along with carbon monitor data.

## Energy Transition & Efficiency (SDG 7)

 	Primary energy consumption (sdg_07_10)
 	Final energy consumption (sdg_07_11)
 	Final energy consumption in households per capita (sdg_07_20)
 	Energy productivity (sdg_07_30)
 	Share of renewable energy in gross final energy consumption by sector (sdg_07_40)

### Energy_consumption

***Final energy consumption in households per capita***

The indicator measures how much electricity and heat every citizen consumes at home excluding energy used for transportation. Since the indicator refers to final energy consumption, only energy used by end consumers is considered. The related consumption of the energy sector itself is excluded.

In [ ]:
url = "https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/data/sdg_07_20/?format=SDMX-CSV"
Energy_consumption = pd.read_csv(url)
print(Energy_consumption.head())


               DATAFLOW        LAST UPDATE freq  unit geo  TIME_PERIOD  \
0  ESTAT:SDG_07_20(1.0)  08/05/25 11:00:00    A  KGOE  AL         2000   
1  ESTAT:SDG_07_20(1.0)  08/05/25 11:00:00    A  KGOE  AL         2001   
2  ESTAT:SDG_07_20(1.0)  08/05/25 11:00:00    A  KGOE  AL         2002   
3  ESTAT:SDG_07_20(1.0)  08/05/25 11:00:00    A  KGOE  AL         2003   
4  ESTAT:SDG_07_20(1.0)  08/05/25 11:00:00    A  KGOE  AL         2004   

   OBS_VALUE OBS_FLAG  CONF_STATUS  
0        123      NaN          NaN  
1        122      NaN          NaN  
2        199      NaN          NaN  
3        125      NaN          NaN  
4        181      NaN          NaN  


In [ ]:
# 1. Keeping only relevant columns
df_clean = Energy_consumption[["geo", "TIME_PERIOD", "OBS_VALUE"]].copy()

# 2. Renaming columns
df_clean.rename(columns={
    "geo": "Country",
    "TIME_PERIOD": "Year",
    "OBS_VALUE": "Energy_consumption"
}, inplace=True)

# 3. Removing any rows where 'value' is missing
df_clean.dropna(subset=["Energy_consumption"], inplace=True)

# 4. Converting columns to correct data types (if needed)
df_clean["Year"] = df_clean["Year"].astype(int)
df_clean["Energy_consumption"] = df_clean["Energy_consumption"].astype(float)

# 5. Filtering for the years 2019–2022
df_clean = df_clean[df_clean["Year"].between(2019, 2022)]

print(df_clean.head())

Energy_consumption = df_clean.copy()

# 7. Saving to individual CSV
#Energy_consumption.to_csv("Energy_consumption.csv", index=False)

   Country  Year  Energy_consumption
19      AL  2019               177.0
20      AL  2020               190.0
21      AL  2021               195.0
22      AL  2022               189.0
42      AT  2019               753.0


### Renewable_Share

***Share of renewable energy in gross final energy consumption by sector***

The indicator measures the share of renewable energy consumption in gross final energy consumption according to the Renewable Energy Directive. The gross final energy consumption is the energy used by end-consumers (final energy consumption) plus grid losses and self-consumption of power plants.

In [ ]:
url = "https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/data/sdg_07_40/?format=SDMX-CSV"
Renewable_Share = pd.read_csv(url)
print(Renewable_Share.head())


               DATAFLOW        LAST UPDATE freq nrg_bal unit geo  TIME_PERIOD  \
0  ESTAT:SDG_07_40(1.0)  07/03/25 23:00:00    A     REN   PC  AL         2004   
1  ESTAT:SDG_07_40(1.0)  07/03/25 23:00:00    A     REN   PC  AL         2005   
2  ESTAT:SDG_07_40(1.0)  07/03/25 23:00:00    A     REN   PC  AL         2006   
3  ESTAT:SDG_07_40(1.0)  07/03/25 23:00:00    A     REN   PC  AL         2007   
4  ESTAT:SDG_07_40(1.0)  07/03/25 23:00:00    A     REN   PC  AL         2008   

   OBS_VALUE  OBS_FLAG  CONF_STATUS  
0     29.620       NaN          NaN  
1     31.367       NaN          NaN  
2     32.070       NaN          NaN  
3     32.657       NaN          NaN  
4     32.448       NaN          NaN  


In [ ]:
df_clean = Renewable_Share[["geo", "TIME_PERIOD", "OBS_VALUE"]].copy()

df_clean.rename(columns={
    "geo": "Country",
    "TIME_PERIOD": "Year",
    "OBS_VALUE": "Renewable_Share"
}, inplace=True)

df_clean.dropna(subset=["Renewable_Share"], inplace=True)
df_clean["Year"] = df_clean["Year"].astype(int)
df_clean["Renewable_Share"] = df_clean["Renewable_Share"].astype(float)
df_clean = df_clean[df_clean["Year"].between(2019, 2022)]
print(df_clean.head())

Renewable_Share = df_clean.copy()
#Renewable_Share.to_csv("Renewable_Share.csv", index=False)

   Country  Year  Renewable_Share
15      AL  2019           38.042
16      AL  2020           45.015
17      AL  2021           41.389
18      AL  2022           44.076
35      AT  2019           33.755


## Economic & Innovation Dimensions (SDG 8 and SDG 9)

### GDP_Capita

***Real GDP per capita (sdg_08_10)***

The indicator is calculated as the ratio of real GDP to the average population of a specific year. GDP measures the value of total final output of goods and services produced by an economy within a certain period of time. It includes goods and services that have markets (or which could have markets) and products which are produced by general government and non-profit institutions. It is a measure of economic activity and is also used as a proxy for the development in a country’s material living standards. However, it is a limited measure of economic welfare. For example, neither does GDP include most unpaid household work nor does GDP take account of negative effects of economic activity, like environmental degradation.

In [ ]:
url = "https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/data/sdg_08_10/?format=SDMX-CSV"
GDP_Capita = pd.read_csv(url)
print(GDP_Capita.head())


               DATAFLOW        LAST UPDATE freq           unit na_item geo  \
0  ESTAT:SDG_08_10(1.0)  02/05/25 23:00:00    A  CLV20_EUR_HAB    B1GQ  AL   
1  ESTAT:SDG_08_10(1.0)  02/05/25 23:00:00    A  CLV20_EUR_HAB    B1GQ  AL   
2  ESTAT:SDG_08_10(1.0)  02/05/25 23:00:00    A  CLV20_EUR_HAB    B1GQ  AL   
3  ESTAT:SDG_08_10(1.0)  02/05/25 23:00:00    A  CLV20_EUR_HAB    B1GQ  AL   
4  ESTAT:SDG_08_10(1.0)  02/05/25 23:00:00    A  CLV20_EUR_HAB    B1GQ  AL   

   TIME_PERIOD  OBS_VALUE OBS_FLAG  CONF_STATUS  
0         2000     2120.0      NaN          NaN  
1         2001     2330.0      NaN          NaN  
2         2002     2440.0      NaN          NaN  
3         2003     2580.0      NaN          NaN  
4         2004     2730.0      NaN          NaN  


In [ ]:
df_clean = GDP_Capita[["geo", "TIME_PERIOD", "OBS_VALUE"]].copy()

df_clean.rename(columns={
    "geo": "Country",
    "TIME_PERIOD": "Year",
    "OBS_VALUE": "GDP_Capita"
}, inplace=True)

df_clean.dropna(subset=["GDP_Capita"], inplace=True)
df_clean["Year"] = df_clean["Year"].astype(int)
df_clean["GDP_Capita"] = df_clean["GDP_Capita"].astype(float)
df_clean = df_clean[df_clean["Year"].between(2019, 2022)]
print(df_clean.head())

GDP_Capita = df_clean.copy()
#GDP_Capita.to_csv("GDP_Capita.csv", index=False)

   Country  Year  GDP_Capita
19      AL  2019      4850.0
20      AL  2020      4710.0
21      AL  2021      5190.0
22      AL  2022      5500.0
42      AT  2019     45730.0


Gross domestic expenditure on R&D by sector (sdg_09_10)
 	Share of buses and trains in inland passenger transport (sdg_09_50)
 	Share of rail and inland waterways in inland freight transport (sdg_09_60)
 	Air emission intensity from industry (sdg_09_70)


### Passenger_transport

***Share of buses and trains in inland passenger transport***

The indicator measures the share of collective transport modes in total inland passenger transport performance, expressed in passenger-kilometres (pkm). Collective transport modes refer to buses, including coaches and trolley-buses, and trains. Total inland transport includes transport by passenger cars, buses and coaches, and trains. All data are based on movements within national territories, regardless of the nationality of the vehicle. The data collection methodology is voluntary and not fully harmonised at the EU level. Other collective transport modes, such as tram and metro systems, are also not included due to the lack of harmonised data. For countries, where rail transport statistical legislation does not apply, the totals contain only the share of coaches, buses and trolley buses.

In [ ]:
url = "https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/data/sdg_09_50/?format=SDMX-CSV"
Passenger_transport = pd.read_csv(url)
print(Passenger_transport.head())


               DATAFLOW        LAST UPDATE freq unit  vehicle geo  \
0  ESTAT:SDG_09_50(1.0)  25/07/24 23:00:00    A   PC  BUS_TOT  AT   
1  ESTAT:SDG_09_50(1.0)  25/07/24 23:00:00    A   PC  BUS_TOT  AT   
2  ESTAT:SDG_09_50(1.0)  25/07/24 23:00:00    A   PC  BUS_TOT  AT   
3  ESTAT:SDG_09_50(1.0)  25/07/24 23:00:00    A   PC  BUS_TOT  AT   
4  ESTAT:SDG_09_50(1.0)  25/07/24 23:00:00    A   PC  BUS_TOT  AT   

   TIME_PERIOD  OBS_VALUE OBS_FLAG  CONF_STATUS  
0         2000       11.0      NaN          NaN  
1         2001       10.9      NaN          NaN  
2         2002       10.9      NaN          NaN  
3         2003       10.9      NaN          NaN  
4         2004       11.0      NaN          NaN  


In [ ]:
df_clean = Passenger_transport[["geo", "TIME_PERIOD", "OBS_VALUE"]].copy()
df_clean.rename(columns={
    "geo": "Country",
    "TIME_PERIOD": "Year",
    "OBS_VALUE": "Passenger_transport"
}, inplace=True)
df_clean.dropna(subset=["Passenger_transport"], inplace=True)
df_clean["Year"] = df_clean["Year"].astype(int)
df_clean["Passenger_transport"] = df_clean["Passenger_transport"].astype(float)
df_clean = df_clean[df_clean["Year"].between(2019, 2022)]
print(df_clean.head())

Passenger_transport = df_clean.copy()
#Passenger_transport.to_csv("Passenger_transport.csv", index=False)

   Country  Year  Passenger_transport
19      AT  2019                  9.7
20      AT  2020                  9.0
21      AT  2021                  9.1
22      AT  2022                  8.4
42      BE  2019                 10.4


### Air_emission_intensity

***Air emission intensity from industry***

This indicator measures the emissions intensity of fine particulate matter (PM2.5) from the manufacturing sector (NACE Rev. 2 sector ‘C’). Fine and coarse particulates (PM10) are less than 10 micrometres in diameter and can be carried deep into the lungs, where they can cause inflammation and exacerbate the condition of people suffering from heart and lung diseases. Fine particulates (PM2.5) are less than 2.5 micrometres in diameter and are therefore a subset of the PM10 particles. Their negative health impacts are more serious than PM10 because they can be drawn further into the lungs and may be more toxic. Emission intensity is calculated by dividing the sector’s PM emissions by its gross value added (GVA), which is defined as output (at basic prices) minus intermediate consumption (at purchaser prices).  

In [ ]:
url = "https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/data/sdg_09_70/?format=SDMX-CSV"
Air_emission_intensity = pd.read_csv(url)
print(Air_emission_intensity.head())


               DATAFLOW        LAST UPDATE freq airpol nace_r2 na_item  \
0  ESTAT:SDG_09_70(1.0)  13/12/24 11:00:00    A   PM10       C     B1G   
1  ESTAT:SDG_09_70(1.0)  13/12/24 11:00:00    A   PM10       C     B1G   
2  ESTAT:SDG_09_70(1.0)  13/12/24 11:00:00    A   PM10       C     B1G   
3  ESTAT:SDG_09_70(1.0)  13/12/24 11:00:00    A   PM10       C     B1G   
4  ESTAT:SDG_09_70(1.0)  13/12/24 11:00:00    A   PM10       C     B1G   

          unit geo  TIME_PERIOD  OBS_VALUE OBS_FLAG  CONF_STATUS  
0  G_EUR_CLV10  AT         2008       0.07      NaN          NaN  
1  G_EUR_CLV10  AT         2009       0.07      NaN          NaN  
2  G_EUR_CLV10  AT         2010       0.07      NaN          NaN  
3  G_EUR_CLV10  AT         2011       0.07      NaN          NaN  
4  G_EUR_CLV10  AT         2012       0.06      NaN          NaN  


In [ ]:
df_clean = Air_emission_intensity[["geo", "TIME_PERIOD", "OBS_VALUE"]].copy()
df_clean.rename(columns={
    "geo": "Country",
    "TIME_PERIOD": "Year",
    "OBS_VALUE": "Air_emission_intensity"
}, inplace=True)
df_clean.dropna(subset=["Air_emission_intensity"], inplace=True)
df_clean["Year"] = df_clean["Year"].astype(int)
df_clean["Air_emission_intensity"] = df_clean["Air_emission_intensity"].astype(float)
df_clean = df_clean[df_clean["Year"].between(2019, 2022)]
print(df_clean.head())

Air_emission_intensity = df_clean.copy()
#Air_emission_intensity.to_csv("Air_emission_intensity.csv", index=False)

   Country  Year  Air_emission_intensity
11      AT  2019                    0.04
12      AT  2020                    0.04
13      AT  2021                    0.04
14      AT  2022                    0.04
26      BE  2019                    0.11


## Sustainable Consumption & Circular Economy (SDG 12)

### Grossvalue_Egoods

***Gross value added in environmental goods and services sector (sdg_12_61)***

The environmental goods and services sector (EGSS) is defined as that part of a country’s economy that is engaged in producing goods and services that are used in environmental protection and resource management activities either domestically or abroad. Gross value added in EGSS represents the contribution of the environmental goods and services sector to GDP and is defined as the difference between the value of the sector’s output and intermediate consumption.

In [ ]:
url = "https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/data/sdg_12_61/?format=SDMX-CSV"
Grossvalue_Egoods = pd.read_csv(url)
print(Grossvalue_Egoods.head())


               DATAFLOW        LAST UPDATE freq nace_r2 ceparema na_item  \
0  ESTAT:SDG_12_61(1.0)  11/04/25 11:00:00    A   TOTAL    TOTAL     B1G   
1  ESTAT:SDG_12_61(1.0)  11/04/25 11:00:00    A   TOTAL    TOTAL     B1G   
2  ESTAT:SDG_12_61(1.0)  11/04/25 11:00:00    A   TOTAL    TOTAL     B1G   
3  ESTAT:SDG_12_61(1.0)  11/04/25 11:00:00    A   TOTAL    TOTAL     B1G   
4  ESTAT:SDG_12_61(1.0)  11/04/25 11:00:00    A   TOTAL    TOTAL     B1G   

         ty        unit geo  TIME_PERIOD  OBS_VALUE OBS_FLAG CONF_STATUS  
0  TOT_EGSS  CLV15_MEUR  AT         2008   15140.19      NaN         NaN  
1  TOT_EGSS  CLV15_MEUR  AT         2009   14099.80      NaN         NaN  
2  TOT_EGSS  CLV15_MEUR  AT         2010   14560.01      NaN         NaN  
3  TOT_EGSS  CLV15_MEUR  AT         2011   14385.16      NaN         NaN  
4  TOT_EGSS  CLV15_MEUR  AT         2012   14978.06      NaN         NaN  


In [ ]:
df_clean = Grossvalue_Egoods[["geo", "TIME_PERIOD", "OBS_VALUE"]].copy()
df_clean.rename(columns={
    "geo": "Country",
    "TIME_PERIOD": "Year",
    "OBS_VALUE": "Grossvalue_Egoods"
}, inplace=True)
df_clean.dropna(subset=["Grossvalue_Egoods"], inplace=True)
df_clean["Year"] = df_clean["Year"].astype(int)
df_clean["Grossvalue_Egoods"] = df_clean["Grossvalue_Egoods"].astype(float)
df_clean = df_clean[df_clean["Year"].between(2019, 2022)]
print(df_clean.head())

Grossvalue_Egoods = df_clean.copy()
#Grossvalue_Egoods.to_csv("Grossvalue_Egoods.csv", index=False)

   Country  Year  Grossvalue_Egoods
11      AT  2019           17021.48
12      AT  2020           16368.20
13      AT  2021           17181.54
14      AT  2022           18634.79
20      BE  2019            7957.43


## Climate Vulnerability & Adaptation (SDG 13)

Climate related economic losses (sdg_13_40)
Contribution to the international 100bn USD commitment on climate related expending (source: DG CLIMA, EIONET) (sdg_13_50)
Green bond issuance by corporates and governments (sdg_13_70a)


### Climate_Losses

***Climate related economic losses***

The indicator measures the economic losses from weather and climate-related events. In addition to the annual figures, a smoothed time-series based on 30-year averages is presented. In line with the climate normal period as defined by World Meteorological Organisation, these 30 years average figures reflect trends excluding the substantial climate variability on shorter time scales due to natural factors. The indicator is based on data from CATDAT of RiskLayer.

In [ ]:
url = "https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/data/sdg_13_40/?format=SDMX-CSV"
Climate_Losses = pd.read_csv(url)
print(Climate_Losses.head())


               DATAFLOW        LAST UPDATE freq statinfo        unit stk_flow  \
0  ESTAT:SDG_13_40(1.0)  06/11/24 23:00:00    A  AVG_30Y  EUR_HAB_KP     LOSS   
1  ESTAT:SDG_13_40(1.0)  06/11/24 23:00:00    A  AVG_30Y  EUR_HAB_KP     LOSS   
2  ESTAT:SDG_13_40(1.0)  06/11/24 23:00:00    A  AVG_30Y  EUR_HAB_KP     LOSS   
3  ESTAT:SDG_13_40(1.0)  06/11/24 23:00:00    A  AVG_30Y  EUR_HAB_KP     LOSS   
4  ESTAT:SDG_13_40(1.0)  06/11/24 23:00:00    A  AVG_30Y  EUR_HAB_KP     LOSS   

  geo  TIME_PERIOD  OBS_VALUE  OBS_FLAG  CONF_STATUS  
0  AT         2009      38.95       NaN          NaN  
1  AT         2010      38.85       NaN          NaN  
2  AT         2011      38.72       NaN          NaN  
3  AT         2012      38.59       NaN          NaN  
4  AT         2013      42.34       NaN          NaN  


In [ ]:
df_clean = Climate_Losses[["geo", "TIME_PERIOD", "OBS_VALUE"]].copy()
df_clean.rename(columns={
    "geo": "Country",
    "TIME_PERIOD": "Year",
    "OBS_VALUE": "Climate_Losses"
}, inplace=True)
df_clean.dropna(subset=["Climate_Losses"], inplace=True)
df_clean["Year"] = df_clean["Year"].astype(int)
df_clean["Climate_Losses"] = df_clean["Climate_Losses"].astype(float)
df_clean = df_clean[df_clean["Year"].between(2019, 2022)]
print(df_clean.head())

Climate_Losses = df_clean.copy()
#Climate_Losses.to_csv("Climate_Losses.csv", index=False)

   Country  Year  Climate_Losses
10      AT  2019           47.20
11      AT  2020           45.88
12      AT  2021           48.13
13      AT  2022           48.46
25      BE  2019           12.64


### Climate_Spending

***Contribution to the international 100bn USD commitment on climate related expending (source: DG CLIMA, EIONET)***

The indicator measures the total amount spent from the annual budget of the EU Member States as well as of the European Commission and the European Investment Bank, in order to contribute to the international 100bn USD commitment for climate finance under the United Nations Framework Convention on Climate Change (UNFCCC). The data for 2020 and 2021 covers commitments for both multilateral and bilateral public finance. From 2022 onwards, the data covers bilateral finance committed and multilateral finance provided.

In [ ]:
url = "https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/data/sdg_13_50/?format=SDMX-CSV"
Climate_Spending = pd.read_csv(url)
print(Climate_Spending.head())


               DATAFLOW        LAST UPDATE freq     unit geo  TIME_PERIOD  \
0  ESTAT:SDG_13_50(1.0)  05/02/25 23:00:00    A  MIO_EUR  AT         2014   
1  ESTAT:SDG_13_50(1.0)  05/02/25 23:00:00    A  MIO_EUR  AT         2015   
2  ESTAT:SDG_13_50(1.0)  05/02/25 23:00:00    A  MIO_EUR  AT         2016   
3  ESTAT:SDG_13_50(1.0)  05/02/25 23:00:00    A  MIO_EUR  AT         2017   
4  ESTAT:SDG_13_50(1.0)  05/02/25 23:00:00    A  MIO_EUR  AT         2018   

   OBS_VALUE OBS_FLAG  CONF_STATUS  
0     141.27      NaN          NaN  
1     117.62      NaN          NaN  
2     199.26      NaN          NaN  
3     164.14      NaN          NaN  
4     239.47      NaN          NaN  


In [ ]:
df_clean = Climate_Spending[["geo", "TIME_PERIOD", "OBS_VALUE"]].copy()
df_clean.rename(columns={
    "geo": "Country",
    "TIME_PERIOD": "Year",
    "OBS_VALUE": "Climate_Spending"
}, inplace=True)
df_clean.dropna(subset=["Climate_Spending"], inplace=True)
df_clean["Year"] = df_clean["Year"].astype(int)
df_clean["Climate_Spending"] = df_clean["Climate_Spending"].astype(float)
df_clean = df_clean[df_clean["Year"].between(2019, 2022)]
print(df_clean.head())

Climate_Spending = df_clean.copy()
#Climate_Spending.to_csv("Climate_Spending.csv", index=False)

   Country  Year  Climate_Spending
5       AT  2019            332.82
6       AT  2020            257.95
7       AT  2021            248.61
8       AT  2022            401.29
15      BE  2019             99.71


### Green_Bonds

***Green bond issuance by corporates and governments***

Green bonds are loans provided by an investor to a borrower which are used to fund projects or activities that promote climate change mitigation or adaptation or other environmental objectives. While the green bond definition can vary, this indicator includes bonds that are aligned with the four core components of the International Capital Market Association (ICMA) green bond principles or are certified by the Climate Bond Initiative (CBI). Issuers include supranational issuers such as the EU or the EIB, subnational issuers such as municipalities or agencies, corporates such as a company or financial corporation, and sovereign bond issuers which are national governments.

In [ ]:
url = "https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/data/sdg_13_70a/?format=SDMX-CSV"
Green_Bonds = pd.read_csv(url)
print(Green_Bonds.head())

                DATAFLOW        LAST UPDATE freq    unit geo  TIME_PERIOD  \
0  ESTAT:SDG_13_70A(1.0)  31/03/25 23:00:00    A  PC_TOT  AT         2014   
1  ESTAT:SDG_13_70A(1.0)  31/03/25 23:00:00    A  PC_TOT  AT         2015   
2  ESTAT:SDG_13_70A(1.0)  31/03/25 23:00:00    A  PC_TOT  AT         2016   
3  ESTAT:SDG_13_70A(1.0)  31/03/25 23:00:00    A  PC_TOT  AT         2017   
4  ESTAT:SDG_13_70A(1.0)  31/03/25 23:00:00    A  PC_TOT  AT         2018   

   OBS_VALUE  OBS_FLAG  CONF_STATUS  
0       0.74       NaN          NaN  
1       0.00       NaN          NaN  
2       1.52       NaN          NaN  
3       0.79       NaN          NaN  
4       0.56       NaN          NaN  


In [ ]:
df_clean = Green_Bonds[["geo", "TIME_PERIOD", "OBS_VALUE"]].copy()
df_clean.rename(columns={
    "geo": "Country",
    "TIME_PERIOD": "Year",
    "OBS_VALUE": "Green_Bonds"
}, inplace=True)
df_clean.dropna(subset=["Green_Bonds"], inplace=True)
df_clean["Year"] = df_clean["Year"].astype(int)
df_clean["Green_Bonds"] = df_clean["Green_Bonds"].astype(float)
df_clean = df_clean[df_clean["Year"].between(2019, 2022)]
print(df_clean.head())

Green_Bonds = df_clean.copy()
#Green_Bonds.to_csv("Green_Bonds.csv", index=False)

   Country  Year  Green_Bonds
5       AT  2019         2.40
6       AT  2020         1.61
7       AT  2021         2.96
8       AT  2022         8.45
15      BE  2019         0.00


# Final Combined Dataset

Now all the datasets shall be combied together for the time period 2019-2022

Carbon_Monitor +

* Energy_consumption +
* Renewable_Share +
* GDP_Capita +
* Passenger_transport +
* Air_emission_intensity +
* Grossvalue_Egoods +
* Climate_Losses +
* Climate_Spending +
*  Green_Bonds

In [ ]:
# Droping aggregate row of "EU27 & UK"
Carbon_Monitor = Carbon_Monitor[Carbon_Monitor["country"] != "EU27 & UK"]

# Filtering Carbon Monitor data for years 2019 to 2022
Carbon_Monitor = Carbon_Monitor[Carbon_Monitor["year"].between(2019, 2022)]

# Pivoting Carbon Monitor data so that each sector's "MtCO2 per year" becomes its own column.
cm_pivot = Carbon_Monitor.pivot_table(
    index=["country", "year"],
    columns="sector",
    values="MtCO2 per year"
).reset_index()

# Grouping and extract total emissions for each country-year.
total_cm = Carbon_Monitor.groupby(["country", "year"])["Total MtCO2 per year"].first().reset_index()
total_cm.rename(columns={"Total MtCO2 per year": "Total_MtCO2"}, inplace=True)

# Merging pivoted sector data with total emissions.
Carbon_Monitor_wide = pd.merge(cm_pivot, total_cm, on=["country", "year"])

# Renaming columns for consistency (full country names are already used here).
Carbon_Monitor_wide.rename(columns={"country": "Country", "year": "Year"}, inplace=True)

# Seting index and group by to ensure uniqueness
Carbon_Monitor_wide.set_index(["Country", "Year"], inplace=True)
Carbon_Monitor_wide = Carbon_Monitor_wide.groupby(level=["Country", "Year"]).first()

In [ ]:
print(Carbon_Monitor_wide.head())

              Domestic Aviation  Ground Transport   Industry  \
Country Year                                                   
Austria 2019           0.052956         24.289183  20.204606   
        2020           0.043006         24.062859  19.251314   
        2021           0.053115         23.814914  24.915473   
        2022           0.032175         23.538949  24.193196   
Belgium 2019           0.007597         24.653963  26.756724   

              International Aviation      Power  Residential  Total_MtCO2  
Country Year                                                               
Austria 2019                2.852046  14.024953     8.880186    70.303931  
        2020                1.018447  12.556342     9.019489    65.951457  
        2021                1.169484  12.950656     9.759080    72.662722  
        2022                1.998732  14.117581     8.826311    72.706944  
Belgium 2019                4.351530  18.312414    24.641355    98.723583  


***Combining All other datasets***

In [ ]:
country_mapping = {
    "AT": "Austria",
    "BE": "Belgium",
    "BG": "Bulgaria",
    "HR": "Croatia",
    "CY": "Cyprus",
    "CZ": "Czech Republic",  # Ensure consistency: if we prefer "Czechia", rename Carbon Monitor accordingly.
    "DK": "Denmark",
    "EE": "Estonia",
    "FI": "Finland",
    "FR": "France",
    "DE": "Germany",
    "EL": "Greece",          # Eurostat often uses "EL" for Greece.
    "HU": "Hungary",
    "IE": "Ireland",
    "IT": "Italy",
    "LV": "Latvia",
    "LT": "Lithuania",
    "LU": "Luxembourg",
    "MT": "Malta",
    "NL": "Netherlands",
    "PL": "Poland",
    "PT": "Portugal",
    "RO": "Romania",
    "SK": "Slovakia",
    "SI": "Slovenia",
    "ES": "Spain",
    "SE": "Sweden",
    "NO": "Norway",
    "CH": "Switzerland",
    "UK": "United Kingdom"
}

In [ ]:
# Geting valid countries from Carbon Monitor data:
valid_countries = set(Carbon_Monitor_wide.index.get_level_values("Country").unique())

In [ ]:
# List of dataset variable names.
dataset_names = [
    "Energy_consumption",
    "Renewable_Share",
    "GDP_Capita",
    "Passenger_transport",
    "Air_emission_intensity",
    "Grossvalue_Egoods",
    "Climate_Losses",
    "Climate_Spending",
    "Green_Bonds"
]

# Process each dataset:
for ds_name in dataset_names:
    df = globals()[ds_name]  # Retrieve the DataFrame.
    # Map abbreviated country codes to full names.
    df["Country"] = df["Country"].map(country_mapping)
    # Drop rows where mapping resulted in NaN.
    df.dropna(subset=["Country"], inplace=True)
    # Filter for years 2019 to 2022.
    df = df[df["Year"].between(2019, 2022)].copy()
    # Keep only rows where Country appears in Carbon Monitor data.
    df = df[df["Country"].isin(valid_countries)].copy()
    # Set index to ["Country", "Year"] and group to ensure uniqueness.
    df.set_index(["Country", "Year"], inplace=True)
    df = df.groupby(level=["Country", "Year"]).first()
    # Update the global variable.
    globals()[ds_name] = df

In [ ]:
GDP_Capita.head()

GDP_Capita
Country Year            
Austria 2019     45730.0
        2020     42650.0
        2021     44520.0
        2022     46350.0
Belgium 2019     42400.0

In [ ]:
dfs_to_merge = [Carbon_Monitor_wide] + [globals()[ds_name] for ds_name in dataset_names]

final_df = pd.concat(dfs_to_merge, axis=1)

# Fill any missing values with zero.
final_df.fillna(0, inplace=True)

# Reset the index to restore "Country" and "Year" as regular columns.
final_df.reset_index(inplace=True)

print(final_df.head())

   Country  Year  Domestic Aviation  Ground Transport   Industry  \
0  Austria  2019           0.052956         24.289183  20.204606   
1  Austria  2020           0.043006         24.062859  19.251314   
2  Austria  2021           0.053115         23.814914  24.915473   
3  Austria  2022           0.032175         23.538949  24.193196   
4  Belgium  2019           0.007597         24.653963  26.756724   

   International Aviation      Power  Residential  Total_MtCO2  \
0                2.852046  14.024953     8.880186    70.303931   
1                1.018447  12.556342     9.019489    65.951457   
2                1.169484  12.950656     9.759080    72.662722   
3                1.998732  14.117581     8.826311    72.706944   
4                4.351530  18.312414    24.641355    98.723583   

   Energy_consumption  Renewable_Share  GDP_Capita  Passenger_transport  \
0               753.0           33.755     45730.0                  9.7   
1               781.0           36.545     4

In [ ]:
#final_df.to_csv("EU Composite Dataset.csv", index=False)

In [ ]:
final_df.head()

,Country,Year,Domestic Aviation,Ground Transport,Industry,International Aviation,Power,Residential,Total_MtCO2,Energy_consumption,Renewable_Share,GDP_Capita,Passenger_transport,Air_emission_intensity,Grossvalue_Egoods,Climate_Losses,Climate_Spending,Green_Bonds
0,Austria,2019,0.052956,24.289183,20.204606,2.852046,14.024953,8.880186,70.303931,753.0,33.755,45730.0,9.7,0.04,17021.48,47.20,332.82,2.40
1,Austria,2020,0.043006,24.062859,19.251314,1.018447,12.556342,9.019489,65.951457,781.0,36.545,42650.0,9.0,0.04,16368.20,45.88,257.95,1.61
2,Austria,2021,0.053115,23.814914,24.915473,1.169484,12.950656,9.759080,72.662722,866.0,34.792,44520.0,9.1,0.04,17181.54,48.13,248.61,2.96
3,Austria,2022,0.032175,23.538949,24.193196,1.998732,14.117581,8.826311,72.706944,762.0,34.075,46350.0,8.4,0.04,18634.79,48.46,401.29,8.45
4,Belgium,2019,0.007597,24.653963,26.756724,4.351530,18.312414,24.641355,98.723583,653.0,9.929,42400.0,10.4,0.11,7957.43,12.64,99.71,0.00


This is the dataset we use for composite index building